In [22]:
import os
import cv2
import numpy as np

In [23]:
# Define the dimensions of your input data
img_rows, img_cols, img_depth = 16, 16, 15

# Define the list of action classes
action_classes = ['boxing', 'handclapping', 'handwaving', 'jogging', 'running', 'walking']

# Initialize lists for training data and labels
X_tr = []  # To store video frames
y_tr = []  # To store action class labels


In [24]:
# Loop through each action class
for class_label, action_class in enumerate(action_classes):
    class_directory = f'kth_dataset/{action_class}'

    for vid in os.listdir(class_directory):
        vid = os.path.join(class_directory, vid)
        frames = []

        cap = cv2.VideoCapture(vid)
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(f"Frames per second: {fps}")

        for k in range(img_depth):
            ret, frame = cap.read()
            frame = cv2.resize(frame, (img_rows, img_cols), interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(gray)

        cap.release()

        input = np.array(frames)
        ipt = np.rollaxis(np.rollaxis(input, 2, 0), 2, 0)

        X_tr.append(ipt)
        y_tr.append(class_label)

Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per second: 25.0
Frames per secon

In [25]:
X_tr_array = np.array(X_tr)  # Convert the frames into a NumPy array
y_tr_array = np.array(y_tr)  # Convert the labels into a NumPy array

num_samples = len(X_tr_array)
print(f"Number of samples: {num_samples}")

Number of samples: 599


In [26]:
# Assign labels to each class
label = np.ones((num_samples,), dtype=int)
label[0:100] = 0
label[100:199] = 1
label[199:299] = 2
label[299:399] = 3
label[399:499] = 4
label[499:] = 5

In [27]:
# Create a tuple with training data and labels
train_data = [X_tr_array, label]
(X_train, y_train) = (train_data[0], train_data[1])
print('X_Train shape:', X_train.shape)

train_set = np.zeros((num_samples, 1, img_rows, img_cols, img_depth))

# Fill the train_set with the video frames
for h in range(num_samples):
    train_set[h][0][:][:][:] = X_train[h, :, :, :]

# Define the patch size (number of frames per video)
patch_size = 15

print(train_set.shape, 'train samples')

X_Train shape: (599, 16, 16, 15)
(599, 1, 16, 16, 15) train samples


In [28]:
import keras.backend as K

K.set_image_data_format('channels_last')  # Set the data format to NDHWC


In [29]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation
from keras.optimizers.legacy import RMSprop

In [30]:
batch_size = 2
nb_classes = 6
nb_epoch = 50

In [31]:
# Convert class vectors to binary class matrices
Y_train = to_categorical(y_train, nb_classes)

# Number of convolutional filters to use at each layer
nb_filters = [32, 32]

# Level of pooling to perform at each layer (POOL x POOL)
nb_pool = [3, 3]

# Level of convolution to perform at each layer (CONV x CONV)
nb_conv = [5, 5]

# Pre-processing

train_set = train_set.astype('float32')

# Normalize the data by subtracting the mean and dividing by the maximum value
train_set -= np.mean(train_set)
train_set /= np.max(train_set)

In [32]:
# Create a Sequential model
model = Sequential()

# Add a 3D convolutional layer
model.add(Conv3D(nb_filters[0], kernel_size=(nb_conv[0], nb_conv[0], nb_conv[0]), input_shape=(1, img_rows, img_cols, patch_size), activation='relu',data_format='channels_first'))

# Add a 3D max-pooling layer
model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0]),data_format='channels_first'))

# Add a dropout layer
model.add(Dropout(0.5))

# Flatten the output
model.add(Flatten())

# Add a fully connected layer with 128 units and ReLU activation
model.add(Dense(128, kernel_initializer='normal', activation='relu'))

# Add another dropout layer
model.add(Dropout(0.5))

# Add the output layer with the number of classes and softmax activation
model.add(Dense(nb_classes, kernel_initializer='normal'))
model.add(Activation('softmax'))

# Compile the model with RMSprop optimizer and categorical cross-entropy loss
model.compile(loss='categorical_crossentropy', optimizer=RMSprop())

# Print a summary of the model's architecture
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 32, 12, 12, 11)    4032      
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 32, 4, 4, 3)       0         
 g3D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 32, 4, 4, 3)       0         
                                                                 
 flatten_4 (Flatten)         (None, 1536)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               196736    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                      

In [33]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(train_set, Y_train, test_size=0.2, random_state=4)

# Train the model
hist = model.fit(X_train_new, y_train_new, validation_data=(X_val_new, y_val_new),
                 batch_size=batch_size, epochs=nb_epoch, verbose=1, shuffle=True)

# Evaluate the model
score = model.evaluate(X_val_new, y_val_new, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/sequential_5/conv3d_5/Conv3D/Conv3DBackpropFilterV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/udaykirantentu/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/udaykirantentu/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1041, in launch_instance

  File "/Users/udaykirantentu/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 724, in start

  File "/Users/udaykirantentu/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py"